In [1]:
#pip install -r requirements.txt

In [ ]:
#recombination_rate=1e-8
#mutation_rate=1.25e-8

In [1]:
import stdpopsim
import os
import tskit
import msprime
import numpy as np
import tsinfer
import zarr
import demesdraw
import subprocess
import pyfaidx
import tsdate
import matplotlib.pyplot as plt
from IPython.display import display, SVG
from Bio import bgzf
#display(SVG(ts.draw_svg()))
from OOA_sim import simple_OOA_sim

/Users/jinminli/research/tree_distance/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def dump_ts(ts,save_dir="./simulations/sim1/",vcz_dir="./simulations/sim1/",name="test",contig_id=0):
    os.makedirs(save_dir, exist_ok=True)
    ts.dump(save_dir+name+".trees")
    np.save(f"{vcz_dir+name}-AA.npy", [s.ancestral_state for s in ts.sites()])
    vcf_name = f"{vcz_dir+name}.vcf.gz"
    with bgzf.open(vcf_name, "wt") as f:
        ts.write_vcf(f,contig_id=contig_id)
    subprocess.run(["tabix", vcf_name])
    ret = subprocess.run(
        ["python", "-m", "bio2zarr", "vcf2zarr", "convert", "--force", vcf_name, f"{vcz_dir+name}.vcz"],
        stderr = None,
    )
    if ret.returncode != 0: 
        print(f"[ERROR] bio2zarr failed for {name} with code {ret.returncode}") 
    else: 
        print(f"[OK] bio2zarr finished for {name}")
def infer_ts(vcz_full_name,tree_full_name):
    #load zarr file, ancestral states and infer
    ancestral_states = np.load(f"{vcz_full_name}-AA.npy")
    vdata = tsinfer.VariantData(f"{vcz_full_name}.vcz", ancestral_states)
    inferred_ts=tsinfer.infer(vdata,recombination_rate=1e-8)
    inferred_ts.dump(f"{tree_full_name}-inferred.trees")
    print("tsinfer done")

In [ ]:
def simulate_dump(model,number_individuals,num_replicates,tree_dir,vcz_dir,sequence_length=1e6):
    if isinstance(model, float):
        name = f"Ne{model:.0e}_{2*number_individuals}_{sequence_length:.0e}"
        
        tree_base_path = os.path.join(tree_dir, name)
        vcz_base_path=os.path.join(vcz_dir, name)
        os.makedirs(tree_base_path, exist_ok=True)
        os.makedirs(vcz_base_path, exist_ok=True)
        for i in range(num_replicates):
            tree_replicate_dir = os.path.join(tree_base_path, str(i))+"/"
            vcz_replicate_dir = os.path.join(vcz_base_path, str(i))+"/"
            os.makedirs(tree_replicate_dir, exist_ok=True)
            os.makedirs(vcz_replicate_dir, exist_ok=True)
            ts=msprime.sim_ancestry(samples=number_individuals,population_size=model,sequence_length=sequence_length,recombination_rate=1e-8)
            ts=msprime.sim_mutations(ts,rate=1e-8)
            dump_ts(ts,save_dir=tree_replicate_dir,vcz_dir=vcz_replicate_dir,name=name+"_"+str(i))
            infer_ts(vcz_full_name=vcz_replicate_dir+name+"_"+str(i),tree_full_name=tree_replicate_dir+name+"_"+str(i))
    else:
        assert(isinstance(model, str))
        assert(model=="ooa")
        name=f"ooa_{6*number_individuals}_{sequence_length:.0e}"
    
        tree_base_path = os.path.join(tree_dir, name)
        vcz_base_path=os.path.join(vcz_dir, name)
        os.makedirs(tree_base_path, exist_ok=True)
        os.makedirs(vcz_base_path, exist_ok=True)
        for i in range(num_replicates):
            tree_replicate_dir = os.path.join(tree_base_path, str(i))+"/"
            vcz_replicate_dir = os.path.join(vcz_base_path, str(i))+"/"
            os.makedirs(tree_replicate_dir, exist_ok=True)
            os.makedirs(vcz_replicate_dir, exist_ok=True)
            ts=simple_OOA_sim(number_individuals,sequence_length)
            dump_ts(ts,save_dir=tree_replicate_dir,vcz_dir=vcz_replicate_dir,name=name+"_"+str(i))
            infer_ts(vcz_full_name=vcz_replicate_dir+name+"_"+str(i),tree_full_name=tree_replicate_dir+name+"_"+str(i))

In [15]:
simulate_dump(2e4,50,100,tree_dir="/Users/jinminli/research/tree_distance/simulations/tree/",vcz_dir="/Users/jinminli/research/tree_distance/simulations/vcf/")

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 24.6files/s]
 Explode: 100%|██████████| 4.21k/4.21k [00:00<00:00, 21.1kvars/s]
  Encode: 100%|██████████| 1.24M/1.24M [00:00<00:00, 11.4MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 1.08karray/s]


[OK] bio2zarr finished for Ne2e+04_100_1e+06_0
tsinfer done


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.7files/s]
 Explode: 100%|██████████| 3.94k/3.94k [00:00<00:00, 21.5kvars/s]
  Encode: 100%|██████████| 1.16M/1.16M [00:00<00:00, 12.6MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 873array/s]


[OK] bio2zarr finished for Ne2e+04_100_1e+06_1
tsinfer done


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.5files/s]
 Explode: 100%|██████████| 4.05k/4.05k [00:00<00:00, 21.4kvars/s]
  Encode: 100%|██████████| 1.19M/1.19M [00:00<00:00, 12.0MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 869array/s]


...

[OK] bio2zarr finished for Ne2e+04_100_1e+06_97
tsinfer done


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.9files/s]
 Explode: 100%|██████████| 4.00k/4.00k [00:00<00:00, 19.4kvars/s]
  Encode: 100%|██████████| 1.18M/1.18M [00:00<00:00, 11.6MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 872array/s]


[OK] bio2zarr finished for Ne2e+04_100_1e+06_98
tsinfer done


    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.9files/s]
 Explode: 100%|██████████| 4.12k/4.12k [00:00<00:00, 18.4kvars/s]
  Encode: 100%|██████████| 1.21M/1.21M [00:00<00:00, 11.8MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 873array/s]


[OK] bio2zarr finished for Ne2e+04_100_1e+06_99
tsinfer done


In [4]:
ts1=msprime.sim_ancestry(samples=50,population_size=1e4,sequence_length=1e6,recombination_rate=1e-8)
ts1=msprime.sim_mutations(ts1,rate=1.25e-8)

In [5]:
dir="./simulations/sim1/"
name="sample100_Ne1e4"
dump_ts(ts1,save_dir=dir,name=name)
infer_ts(full_name=dir+name)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 87.8files/s]
 Explode: 100%|██████████| 2.59k/2.59k [00:00<00:00, 17.1kvars/s]
  Encode: 100%|██████████| 762k/762k [00:00<00:00, 11.3MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 861array/s]


[OK] bio2zarr finished for sample100_Ne1e4
tsinfer done


In [6]:
ooa_ts=simple_OOA_sim(50,1e6)
dir="./simulations/ooa_300_1e6/"
name="ooa_300_1e6"
dump_ts(ooa_ts,save_dir=dir,name=name)
infer_ts(full_name=dir+name)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 84.4files/s]
 Explode: 100%|██████████| 4.12k/4.12k [00:00<00:00, 17.7kvars/s]
  Encode: 100%|██████████| 3.27M/3.27M [00:00<00:00, 25.8MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 1.05karray/s]


[OK] bio2zarr finished for ooa_300_1e6
tsinfer done


In [7]:
ooa_ts=simple_OOA_sim(500,1e6)
dir="./simulations/ooa_3000_1e6/"
name="ooa_3000_1e6"
dump_ts(ooa_ts,save_dir=dir,name=name)
infer_ts(full_name=dir+name)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.9files/s]
 Explode: 100%|██████████| 8.87k/8.87k [00:01<00:00, 7.02kvars/s]
  Encode: 100%|██████████| 66.9M/66.9M [00:00<00:00, 154MB/s] 
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 872array/s]


[OK] bio2zarr finished for ooa_3000_1e6
tsinfer done


In [8]:
ts1=msprime.sim_ancestry(samples=50,population_size=2e4,sequence_length=1e6,recombination_rate=1e-8)
ts1=msprime.sim_mutations(ts1,rate=1.25e-8)
dir="./simulations/Ne2e4_100_1e6/"
name="Ne2e4_100_1e6"
dump_ts(ts1,save_dir=dir,name=name)
infer_ts(full_name=dir+name)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 98.1files/s]
 Explode: 100%|██████████| 4.95k/4.95k [00:00<00:00, 21.7kvars/s]
  Encode: 100%|██████████| 1.45M/1.45M [00:00<00:00, 11.2MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 873array/s]


[OK] bio2zarr finished for Ne2e4_100_1e6
tsinfer done


In [9]:
ts1=msprime.sim_ancestry(samples=500,population_size=2e4,sequence_length=1e6,recombination_rate=1e-8)
ts1=msprime.sim_mutations(ts1,rate=1.25e-8)
dir="./simulations/Ne2e4_1000_1e6/"
name="Ne2e4_1000_1e6"
dump_ts(ts1,save_dir=dir,name=name)
infer_ts(full_name=dir+name)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 88.9files/s]
 Explode: 100%|██████████| 7.73k/7.73k [00:00<00:00, 10.5kvars/s]
  Encode: 100%|██████████| 19.7M/19.7M [00:00<00:00, 82.0MB/s]
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 873array/s]


[OK] bio2zarr finished for Ne2e4_1000_1e6
tsinfer done


In [10]:
ts1=msprime.sim_ancestry(samples=1500,population_size=2e4,sequence_length=1e6,recombination_rate=1e-8)
ts1=msprime.sim_mutations(ts1,rate=1.25e-8)
dir="./simulations/Ne2e4_3000_1e6/"
name="Ne2e4_3000_1e6"
dump_ts(ts1,save_dir=dir,name=name)
infer_ts(full_name=dir+name)

    Scan: 100%|██████████| 1.00/1.00 [00:00<00:00, 78.6files/s]
 Explode: 100%|██████████| 8.41k/8.41k [00:01<00:00, 5.37kvars/s]
  Encode: 100%|██████████| 63.5M/63.5M [00:00<00:00, 155MB/s] 
Finalise: 100%|██████████| 11.0/11.0 [00:00<00:00, 868array/s]


[OK] bio2zarr finished for Ne2e4_3000_1e6
tsinfer done
